### Recomendação musical em decorrência do mapeamento facial

_Este projeto de pesquisa explora o contexto da Inteligência Artificial para a identificação de microexpressões faciais sutis, de menores dimensões, e involuntárias. Ainda, este projeto utiliza aspectos da musicoterapia, a fim de se viabilizar um sistema de recomendação musical._

### Bibliotecas necessárias
__________

In [1]:
import cv2
import numpy as np
import os
from deepface import DeepFace
import pandas as pd
from genericpath import exists
from pandas import DataFrame
from sklearn.neighbors import NearestNeighbors
import math
import webbrowser
from main import training,get_face_image,GetRosto,get_face_param,intervaloConfianca,play_music,feedback

### Input (vídeo/foto)
_______

In [ ]:
database = pd.read_csv("database/database2.csv")
print(database)

#Treinamento
knn = training(database)

img1 = 'img_database_r\Homem\Pessoa 04\Bravo_r.png'

#Imagem ou Câmera?
pred = get_face_image(GetRosto())
#pred = get_face_image(img1)

### Dedução da emoção predominante
______

In [ ]:
if pred != 0:
     new_data = get_face_param(pred)

rosto = new_data[['age', 'gender', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']]
rosto['gender'] = rosto['gender'].replace('Man',0).replace('Woman',1)
print(rosto)

item_selected = rosto.iloc[[0]]
item_selected = item_selected[rosto.columns] 

### Recomendação musical 

________

In [ ]:
# Determine the neighbors
d, neighbors = knn.kneighbors(item_selected.values.reshape(1, -1))
neighbors = neighbors[0][1::]
d = d[0][1::]

print(neighbors)

neighborsMusics = pd.DataFrame()

for x in neighbors:
    print(database.iloc[[x]][['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                              'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']])
    neighborsMusics = neighborsMusics.append(database.iloc[[x]][['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                                                                 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']])

intervalo_params = intervaloConfianca(neighborsMusics)

music_names = []
artist_names = []
artist_genres = []
for y in neighbors:
    music_names.append(database.iloc[y]['music_name'])
    artist_names.append(database.iloc[y]['artist_name'])
    res = (database.iloc[y]['artist_genres']).strip('][').split(', ')
    for item in res:
        if (not (item.replace("'", '') in artist_genres)):
            artist_genres.append(item.replace("'", ''))
    #artist_genres = [*artist_genres , *(database.iloc[y]['artist_genres'])]
    # print(artist_genres)

print(intervalo_params)
print(music_names)
print(artist_names)
print(artist_genres)

### Spotify Dependences
***

In [ ]:
# Spotify WebAPI

musicaRecomendada = neighborsMusics

musicaRecomendada = neighborsMusics.mean()[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                                            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]

musicaRecomendada[['key']] = musicaRecomendada[['key']].values.round()
musicaRecomendada[['mode']] = musicaRecomendada[['mode']].values.round()
musicaRecomendada['duration_ms'] = 210000

musicaRecomendada = musicaRecomendada.to_frame().T

print(musicaRecomendada)

# Recebe musicId
play_music("2hl6q70unbviGo3g1R7uFx")

feedback(musicaRecomendada, new_data)

### Resposta do usuário

_______

### Cálculo do erro
________